# xSoc Python Course Week 6 
### *Efficiency, Compactness, Readability*

## Recursion means Recursion

One way to make things more compact is through recursion. Put simply, *recursive* functions are defined in terms of themselves. For example, this recursive function calculates the sum of the first $n$ integers.

In [ ]:
# Calculates 1 + 2 + ... + n, recursively
def sum_of_first(n: int):
    # Base Case (will eventually be reached)
    if (n <= 0):
        return 0
    
    # General Case (argument steps down by 1 each time)
    return n + sum_of_first(n-1)

print(sum_of_first(4))
print(sum_of_first(7))
print(sum_of_first(0))

Cool, right? But be careful! Every recursive function must satisfy 3 conditions, or else you're in for a bad time:
- It must have at least one *base case*, some combination of arguments that makes it stop calling itself.
- It must have a *recursive case* that moves you towards reaching the base case.
- The base case must be reached after a *finite* number of function calls!

Too many stacked function calls will lead to a *Stack Overflow* Exception, because our computer has run out of memory.

Recursive functions are a powerful tool, because they allow us to write what might otherwise be a large chunk of code in a compact way, that (depending on what it does) might be easier to understand too.

> Task 1: What do each of the following recursive functions do?

*If you're still not quite sure, feel free to call the function with different values.*

In [ ]:
# Function 1
def f(n: int) -> int:
    if n == 0:
        return n
    return n * f(n-1)

In [19]:
# Function 2
def r(s: str) -> bool:
    if len(s) <= 1:
        return True
    if s[0] != s[-1]:
        return False
    return r(s[1:-1])

In [ ]:
# Function 3
def p(x: int, n: int) -> int:
    if n == 0:
        return 1
    
    result = p(x, n // 2)
    result *= result
    if n % 2 == 1:
        result *= x
    return result

> Task 2: Write a recursive function that takes a list of numbers as input, and multiplies them all together.

*There are easier ways of doing this in practice, this is just to get you used to writing recursive functions.*

In [ ]:
def recursive_product(nums: list) -> int:
    # Base Case?
    if not nums:
        return 0

    # Recursive Case?
    return 404

print(recursive_product(3, 4, 5))  # Should output 60

Looks like you've got the basics of recursion down. Let's discuss one more example before moving on - Fibonnacci numbers.

The first two Fibonnaci numbers are $F(0)=0$ and $F(1) = 1$. From there we have $F(n) = F(n-1)+F(n-2)$ - in other words, the next term in the sequence is the sum of the two previous terms. As you might expect, this is perfect for recursion. Give it a run!

In [ ]:
def fib(n: int) -> int:
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

print(fib(4))
print(fib(20))
print(fib(50))

Hmm, that's strange. The first two function calls printed something out, but the third one didn't. Or at least, that's what it looks like. The truth is, `fib(50)` is still being computed by your machine! Surely that can't be correct, it's not that tricky to add the previous terms together until we reach $F(50)$, right? Look, here's a non-recursive version that handles it just fine:

In [ ]:
def fib_linear(n: int) -> int:
    if n <= 1:
        return n
    prev = 0
    curr = 1

    for _ in range(n-1):
        prev, curr = curr, prev + curr

    return curr

print(fib_linear(50))

So what's really going on here? It's to do with the way that functions are called by Python itself. Some functions might do something slightly different each time they're called, even with the same parameters. Because of this, after Python returns a result from `fib()`, it throws away the connection between the parameters passed and the result returned. 

So, we're not just computing one call of `fib()` for each parameter. We're computing...

***There should be a diagram of the exploding recursive call tree here.***

Oh god.

Okay, so how do we stop Python from doing this? Well, there are two ways. The first is to build up a dictionary of previously seen values, and stop ourselves every time we're about to make an unnecessary call by doing a lookup instead. Not amazing to read, but does the job:

In [ ]:
seen = {}

def fib_dict(n: int) -> int:
    if n <= 1:
        return n
    prev1 = seen[n-1] if n-1 in seen else fib_dict(n-1)
    prev2 = seen[n-2] if n-2 in seen else fib_dict(n-2)
    seen[n] = prev1 + prev2
    return seen[n]

print(fib_dict(50))
print(fib_dict(100))

There's a cooler answer, and it's called the `cache` **decorator**. We're not going to spend too much time discussing decorators, but all you need to know is that it's a fancy function that uses another function as argument (yes, you can do that, sorry). Something to look into after you've finished this course, eh? Behind the scenes we're effectively doing the same as the above method, but it looks a lot cleaner.

In [ ]:
from functools import cache

@cache
def fib_cache(n: int) -> int:
    if n <= 1:
        return n
    return fib_cache(n-1) + fib_cache(n-2)

print(fib_cache(50))
print(fib_cache(100))

Let's put what we've learned into practice with a trickier recursive task.

> You're a pirate, sailing the seven seas and looking for treasure. You've got a treasure map, consisting of a grid split into many different squares. Not only that, but it's marked with the number of buried treasures in each square's region! But alas, the strong tailwinds mean you can only sail to squares south or east of your current location, starting at the most north-west square. Can you find the most bountiful route through these waters, ending at your home port in the very south-east?
>
> ***Example image here for clarity on the task***
> 
> Task: Write a function that takes in a 2D list representing the treasure values on each map grid square, and returns the maximum number of treasures that could be claimed in a single trip across these windy waters...

In [ ]:
def maximum_treasure(grid) -> int:
    # This is just the maximum treasure count at the bottom-right grid square. Return it!
    return max_at_square(grid, len(grid) - 1, len(grid[0]) - 1)

def max_at_square(grid, x: int, y: int) -> int:
    # What makes a coordinate invalid?
    # What should we return in this case?
    if x < 0 or y < 0:
        return 0
    
    # The maximum number of treasures up to and including this square.
    # ...the best of two other best routes, plus itself?
    return grid[x][y] + max(max_at_square(grid, x, y-1), max_at_square(grid, x-1, y))

print(
    maximum_treasure(
        [
            [1, 3, 4], 
            [2, 3, 1], 
            [1, 3, 2]
        ]
    )  # Should return 12 (1 -> 3 -> 3 -> 3 -> 2)
)
# What will happen on much larger lists?
print(maximum_treasure([[1] * 50] * 50))

## Searching for an answer

The most basic way to find an item in a list is using a ***Linear Search***. This just entails using a loop to iterate through each element in the list, one-by-one, until either we find the item we want, or we reach the last item in the list (which isn't the target).

Another searching algorithm is a ***Binary Search***:
- Choose the midpoint of the list as the *pivot*
- If the pivot is the target, return the index of the pivot
- If the pivot is larger than the target, look in the left sublist
- If the pivot is smaller than the target, look in the right sublist
- If you only have one item in the list, and it isn't the target, the target is not in the list

In order to work, the Binary Search can only be used on *ordered* lists. Otherwise, we can't guarantee that when we discard part of the list, we're not discarding the target item. That means you may need to sort the list before searching. Here's a Python implementation (assuming all elements in the list are of the same type):

In [ ]:
def binary_search(arr, target):
    arr.sort()      # make sure the array is sorted
    low = 0     # start of the list
    high = len(arr) - 1     # last index of the list

    while low <= high:    # checks we haven't looked at every element in the list
        mid = (low+high) // 2
        if arr[mid] == target:
            return mid      # we found the item!
        elif arr[mid] < target:
            low = mid+1     # move to the right sublist
        else:
            high = mid - 1      # move to the left sublist
    return False    # item isn't in the list

my_array = [1, 1, 2, 3, 5, 8, 13]
print(binary_search(my_array, 1))       # should be fine
print(binary_search(my_array, 14))      # uh-oh

The Binary Search is *orders* more efficient than the Linear Search. In Big-Oh terms, a Linear Search runs in O(n), whereas the Binary Search runs in O(log(n)). The reason? Think about how the binary search works: if we don't find the element, we discard part of the list. To be exact, since we choose the pivot as the midpoint of the list, we discard *half* of the list. That means the binary search reduces our search space by half... at every iteration. So the expected number of pivots we check is log2(n), where n is the initial size of the array.

Once again, Python has an in-built search function: the `in` keyword:

In [ ]:
my_array = [1, 1, 2, 3, 5, 8, 13]
print(1 in my_array)       # should be fine
print(14 in my_array)      # uh-oh

Since Python supports string indexing, you can also search for substrings within strings (`in` is case-sensitive - the string has to match exactly):

In [ ]:
my_string = "Computing Society!"
print("Comp" in my_string)
print("comp" in my_string)

> Task: I've written the *iterative* implementation of the binary search, now you write the *recursive* implementation.

### A Few Useful Data Structures
A ***set*** is a collection that is 
- Unordered - items can appear in any order
- Unindexed - items cannot be referred to by an index or key
- Unchangable - once a set has been declared, you cannot directly edit an individual element

It also does not allow duplicate values. Just like with lists, sets can contain elements of different types.

The syntax for declaring a set is: `my_set = {"item1", "item2", "item3", ...}`

An alternative way to declare a set is by using the set constructor: `my_constructed_set = set(("first_item", "second_time", ...))`

Note the double brackets here - they're important as they tell Python that all the values we passed in are the values we want to add to the set.

The only way to "change" the members of a set is by using the `.add()` and `.remove()` elements, which do what they say on the tin. Note `.remove()` will return an error if the element you pass in does not appear in the set.

Since items are unindexed, there's no way to directly access them. You can however loop through the *entire* set with a for loop:

In [ ]:
my_set = {1,2,3}

for el in my_set:
    print(el)

Python also supports standard set operations with the methods `.difference()`, `.intersection()`, `.symmetric_difference()` and `.union()`. Using these methods will return a new set, if you just want to update an existing set, use the methods `.difference_update()`, `.intersection_update()`, `.symmetric_difference_update()` and `.update()`. Here's a quick example to show the difference:

In [ ]:
set1 = {"a", "b" , "c"}
set2 = {1, 2, 3}

print(set1)
print(set2)

set3 = set1.union(set2)
print(set3)
set1.update(set2)
print(set1)

> Task: Using the "standard" methods (i.e. `.difference()`, `.intersection()`, etc.) to print out the difference, intersection, symmetric difference and union of the sets containing the first 10 multiples of 3, the first 10 multiples of 4, and the first 10 multiples of 5.
>
> Task: Repeat using the update variations.

### Condensing your code
You may have gathered by now that Python is a big fan of shorthand notations. ***Comprehensions*** cut-down the amount of lines used up when you declare a new list/set/dictionary.

A ***list comprehension*** has the format `new_list = [output for var in input]`, e.g. this snippet declares a list of the numbers 1 to 10 (inclusive): `numbers = [i for i in range(1,11)]`. You can also have `if` statements in the comprehension: `even_numbers = [i for i in range(1,11) if (i % 2 == 0)]` would make a list of all the even numbers between 1 and 10. If you want to include an `else` statement, the condition needs to come first, followed by the for loop: `numbers = ["even" if (i % 2 == 0) else "odd" for i in range(1,11)]`. Note: there's no `elif` statments here - you'll need to put another `if` statement after your `else`.

> Task: using a list comprehension, create a list called `fizz_buzz`. The list should have a length of 50 and follow these rules (here index will start from 1 on the first element):
> - If the index is divisible by 3, the element should be "Fizz"
> - If the index is divisible by 5, the element should be "Buzz"
> - If the index is divisible by 3 *and* 5, the element should be "FizzBuzz"
> - Otherwise, the element should simply be the index.

One of the main issues with using any format of comprehension is that your "simplification" can quickly become very complicated. This can happen especially when nesting list comprehensions *within* list comprehensions, or, like in the `fizz_buzz` task, where you had many conditions to consider. If things do start to get tricky, and you're struggling with how to write the comprehension, it's best to just stick with the long way - otherwise you may introduce an error into your program.

If implementing your own sort is too much effort, don't worry - Python has your back. The inbuilt `sort()` method can be called on any list using `array.sort()`. The method will sort the list in ascending order by default; if you want the list sorted in ascending order you need to specify the `reverse` parameter: `array.sort(reverse=true)`.

> Task: Use `.sort()` to sort `unsorted_array` in *ascending* order. Now sort it in *descending* order.

### A Quick Detour: Efficiency and Big-Oh
If the first bubble sort we wrote worked, then why did we improve it? It has to do with *code efficiency*. Often, there are lots of different implementations that solve the same problem. The best solutions will run the fastest, and use the least amount additional storage space. Many of the constructs you've learned over the past weeks will help with writing efficient code, such as:
- Using loops for repeated actions
- Using data structures instead of separate variables
- Using functions if you're going to be repeating the same blocks of actions throughout your code
- Use of in-built features / external code libraries
- Use of recursion

A fairly common notation you'll see when talking about the efficiency of algorithms is **Big-Oh notation O()**. Big-Oh gives us an upper bound to the growth rate of an algorithm as the input size *n* increases. 
Big-Oh has a couple of basic rules:
- If your run time is a polynomial of degree *d*, then the run time is O(nᵈ). You drop any lower-order and constant terms (as n increases, the nᵈ term grows the fastest)
- Use the *smallest possible* class of function

For example, consider going one by one through a list of *n* elements. As we add more elements to the list, it's going to take longer to visit all of the elements. The time varies linearly (hence the term *linear search*) with input size, so we say it runs in O(n) time.

By contrast, the bubble and insertion sorts we wrote in the previous section run in O(n²) time. That means that the run time of our algorithms grow proportional to the **square** of the size of the input. For smaller inputs, it's not a huge problem, but if were sorting arrays with thousands of elements, things would quickly get out of hand.

Not to say that there aren't *worse* sorting algorithms out there. The ***Bogosort*** has no upper bound on its runtime (aka O(∞)) and an **average** runtime of O((n+1)!). Here's a lovely implementation of it below, and an example list of 12 numbers for it to sort. Go on, give it a run!

In [ ]:
from random import shuffle

def in_order(some_list):
    for i in range(len(some_list)-1):
        if some_list[i] >= some_list[i+1]:
            return False
    return True

def bogosort(some_list):
    while not in_order(some_list):
        shuffle(some_list)

nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1]
bogosort(nums)
print(nums)

Unsuprisingly, nobody actually uses Bogosorts.

### Coding Practices
80% of coding is knowing how to code, the remaining 20% is knowing how to code *nicely*. That means writing your code bearing in mind that someone else might have to read it, and they won't want to spend much effort trying to figure out what you're trying to do (it also helps avoid the "what was I thinking" moments when you go back to debug your code). The general term for confusing, unclear, badly written code is ***Spaghetti Code***. We want to avoid spaghetti code as much as possible, otherwise we end up with code like this:

In [ ]:
def func(a, b):
    if a == 0:
        return b
 
    return func(b % a, a)

# what did I just read

Can you figure out what that function does? You're looking at a *very badly* written implementation of the Euclidean Algorithm (finds the GCD of any two numbers).

To make your code more readable, there's a few things you can do:
- Use meaningful variable names, and be consistent with how you name variables (e.g. camelCasing, snake_casing)
- Have whitespace (empty lines) in your code to split it up
- Use comments to explain what your code is doing; you don't have to comment every line - just ones where its not obvious or you think the line is important. In fact, over-commenting is just as bad as not commenting at all
- Be consistent with your indentation
- Avoid hard-coding values in your code (e.g. if you had an array length 5, just using the number 5 instead of len(array)) - it makes it harder to change your code if your values need to change

### ***This week of content was written by the Computing Society***

We hope you enjoyed the course!